In [ ]:
%cd /root/share/Real/KAIST/CD_Flow/
%load_ext autoreload
%autoreload 2

: 

In [2]:
!pip install git+https://github.com/VLL-HD/FrEIA.git tensorboard pytorch-ignite==0.4.2

  Cloning https://github.com/VLL-HD/FrEIA.git to /tmp/pip-req-build-mvppe1y6
  Running command git clone -q https://github.com/VLL-HD/FrEIA.git /tmp/pip-req-build-mvppe1y6
     |████████████████████████████████| 6.0 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 175 kB 81.8 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 170.8 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 112.8 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 122 kB/s  eta 0:00:01
     |████████████████████████████████| 232 kB 154.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 197.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 105.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.8 MB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 36.7 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 129.7 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 380 kB/s s eta 0:00

In [2]:
import numpy as np
import pandas as pd
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score,  classification_report, f1_score

import FrEIA.framework as Ff
import FrEIA.modules as Fm

import torch
import torch.nn as nn
import torch.nn.functional as F

import model_flow
import constant as const
import utils
import main

In [3]:
df = pd.read_csv("Data/no_alien_processing0_fillNA.csv")

info = {
    0: [df['bmi']>=25, ['bmi', 'height', 'weight', 'waist']],
    1: [df['blood_sugar']>=126, ['blood_sugar']],
    2: [df['neutral_fat']>=np.log1p(200), ['neutral_fat']],
    3: [(df['hdl']<40)|(df['ldl']>=160), ['hdl', 'ldl']],
    4: [(df['got']>np.log1p(40))|(df['gpt']>np.log1p(40)), ['got', 'gpt']],
    5: [df['hemoglobin']+df['gender']<13, ['hemoglobin']],
    6: [(df['max_bp']>=130)|(df['min_bp']>80), ['max_bp', 'min_bp']]
}

df = df.drop(df.columns[:63], axis=1)

In [4]:
def init_seeds(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [5]:
def train(num, random_state):
    init_seeds(random_state)
    num = num

    con_vec = ["age", "gender"]

    diseases = np.where(info[num][0], 1, 0)
    encoder = LabelEncoder()
    diseases = encoder.fit_transform(diseases)

    normal = df[diseases == 0]
    diseases = df[diseases == 1] 
    x_diseases = diseases.drop(columns=info[num][1], axis=1)
    y_diseases = pd.DataFrame(np.ones(len(x_diseases)), columns=['diseases'], index=x_diseases.index)
    x_normal = normal.drop(columns=info[num][1], axis=1)
    y_normal = pd.DataFrame(np.zeros(len(x_normal)), columns=['diseases'], index=x_normal.index)

    x_train, x_test, y_train, y_test = train_test_split(x_normal, y_normal, test_size=0.2, stratify=normal['gender'])
    
    x_test = pd.concat([x_test, x_diseases])
    y_test = pd.concat([y_test, y_diseases])
    
    c_train = torch.FloatTensor(x_train[con_vec].values).to('cuda')
    c_test = torch.FloatTensor(x_test[con_vec].values).to('cuda')


    scaler = RobustScaler()
    x_train[x_train.columns] = scaler.fit_transform(x_train[x_train.columns])
    x_test[x_test.columns] = scaler.transform(x_test[x_test.columns])

    x_train.drop(con_vec, axis=1, inplace=True)
    x_test.drop(con_vec, axis=1, inplace=True)
    
    x_train = torch.FloatTensor(x_train.values).to('cuda')
    #y_train = torch.tensor(y_train.values).to("cuda")
    x_test = torch.FloatTensor(x_test.values).to('cuda')
    #y_test = torch.tensor(y_test)

    model = model_flow.CD_Flow(
        dim_features = x_train.shape[1],
        flow_steps = const.FLOW_STEPS,
        cond_dims = len(con_vec)
    ).to('cuda')

    main.train(model, x_train, x_test, y_test, c_train, c_test, multiple = True)

In [6]:
import shutil
try:
    shutil.rmtree('_experiments')
except:
    pass

for ran in range(42, 92): #42, 43 .... , 51
    print("\n\n------RANDOM STATE:", ran, "------")
    train(1, ran)



------RANDOM STATE: 42 ------


2023-01-17 09:16:42,519 eval INFO: [epoch 0] [AUROC 0.8295517432208077] [AUPRC 0.10145685883187043]
2023-01-17 09:16:43,018 eval INFO: [epoch 1] [AUROC 0.8367459878251247] [AUPRC 0.10075027291314873]
2023-01-17 09:16:43,514 eval INFO: [epoch 2] [AUROC 0.8334255672385166] [AUPRC 0.1015655111060602]
2023-01-17 09:16:44,013 eval INFO: [epoch 3] [AUROC 0.8312119535141121] [AUPRC 0.10666261476911584]
2023-01-17 09:16:44,513 eval INFO: [epoch 4] [AUROC 0.8262313226342005] [AUPRC 0.10779225666909083]
2023-01-17 09:16:45,014 eval INFO: [epoch 5] [AUROC 0.8223574986164907] [AUPRC 0.10428364730557137]
2023-01-17 09:16:45,513 eval INFO: [epoch 6] [AUROC 0.8195904814609845] [AUPRC 0.10194356008462947]
2023-01-17 09:16:46,012 eval INFO: [epoch 7] [AUROC 0.8146098505810734] [AUPRC 0.0985785468056503]
2023-01-17 09:16:46,510 eval INFO: [epoch 8] [AUROC 0.8135030437188715] [AUPRC 0.09818975385851232]
2023-01-17 09:16:47,011 eval INFO: [epoch 9] [AUROC 0.8129496402877696] [AUPRC 0.09753928244519808]
20



------RANDOM STATE: 43 ------


2023-01-17 09:18:20,928 eval INFO: [epoch 0] [AUROC 0.8013281682346428] [AUPRC 0.0785150765351278]
2023-01-17 09:18:21,431 eval INFO: [epoch 1] [AUROC 0.8118428334255673] [AUPRC 0.11398730058855172]
2023-01-17 09:18:21,937 eval INFO: [epoch 2] [AUROC 0.822080796900941] [AUPRC 0.12283912976453272]
2023-01-17 09:18:22,434 eval INFO: [epoch 3] [AUROC 0.8254012174875485] [AUPRC 0.14816921029597932]
2023-01-17 09:18:22,936 eval INFO: [epoch 4] [AUROC 0.8284449363586054] [AUPRC 0.18725918670004105]
2023-01-17 09:18:23,434 eval INFO: [epoch 5] [AUROC 0.8289983397897066] [AUPRC 0.1888757345535585]
2023-01-17 09:18:23,933 eval INFO: [epoch 6] [AUROC 0.835085777531821] [AUPRC 0.1942947763467595]
2023-01-17 09:18:24,431 eval INFO: [epoch 7] [AUROC 0.8378527946873271] [AUPRC 0.19606740470743392]
2023-01-17 09:18:24,927 eval INFO: [epoch 8] [AUROC 0.8406198118428335] [AUPRC 0.19920751950030655]
2023-01-17 09:18:25,426 eval INFO: [epoch 9] [AUROC 0.8433868289983399] [AUPRC 0.20482414895935658]
2023-



------RANDOM STATE: 44 ------


...



------RANDOM STATE: 55 ------


2023-01-17 09:37:55,754 eval INFO: [epoch 0] [AUROC 0.8804648588821254] [AUPRC 0.12479980930416439]
2023-01-17 09:37:56,249 eval INFO: [epoch 1] [AUROC 0.8887659103486444] [AUPRC 0.21243724024956395]
2023-01-17 09:37:56,749 eval INFO: [epoch 2] [AUROC 0.8954067515218592] [AUPRC 0.2850192456481714]
2023-01-17 09:37:57,245 eval INFO: [epoch 3] [AUROC 0.8987271721084668] [AUPRC 0.311901249514897]
2023-01-17 09:37:57,745 eval INFO: [epoch 4] [AUROC 0.9042612064194793] [AUPRC 0.35001090733736223]
2023-01-17 09:37:58,243 eval INFO: [epoch 5] [AUROC 0.9131156613171002] [AUPRC 0.43897276442268207]
2023-01-17 09:37:58,743 eval INFO: [epoch 6] [AUROC 0.9164360819037081] [AUPRC 0.44378600776810434]
2023-01-17 09:37:59,241 eval INFO: [epoch 7] [AUROC 0.9169894853348101] [AUPRC 0.44400390753781]
2023-01-17 09:37:59,733 eval INFO: [epoch 8] [AUROC 0.920863309352518] [AUPRC 0.4431542445257244]
2023-01-17 09:38:00,250 eval INFO: [epoch 9] [AUROC 0.9236303265080243] [AUPRC 0.44538634788995424]
2023-01-



------RANDOM STATE: 56 ------


2023-01-17 09:39:33,507 eval INFO: [epoch 0] [AUROC 0.8868289983397891] [AUPRC 0.21391373025788385]
2023-01-17 09:39:34,011 eval INFO: [epoch 1] [AUROC 0.8887659103486442] [AUPRC 0.189377096799166]
2023-01-17 09:39:34,512 eval INFO: [epoch 2] [AUROC 0.8998339789706691] [AUPRC 0.20595723381544692]
2023-01-17 09:39:35,007 eval INFO: [epoch 3] [AUROC 0.8920863309352519] [AUPRC 0.19768719899243412]
2023-01-17 09:39:35,507 eval INFO: [epoch 4] [AUROC 0.8810182623132263] [AUPRC 0.1877954368909614]
2023-01-17 09:39:36,003 eval INFO: [epoch 5] [AUROC 0.8710570005534033] [AUPRC 0.1721434275628615]
2023-01-17 09:39:36,503 eval INFO: [epoch 6] [AUROC 0.8716104039845045] [AUPRC 0.1758400618653207]
2023-01-17 09:39:37,002 eval INFO: [epoch 7] [AUROC 0.8699501936912011] [AUPRC 0.17380706228887013]
2023-01-17 09:39:37,500 eval INFO: [epoch 8] [AUROC 0.8727172108467073] [AUPRC 0.17936906090242857]
2023-01-17 09:39:37,998 eval INFO: [epoch 9] [AUROC 0.8738240177089108] [AUPRC 0.17189996647953526]
2023-

KeyboardInterrupt: 